# SeisBench Testcase Prediction (Detection/Picking) On Earthquake Swarms in the PNSN Network
This notebook provides a worked example of using pre-trained ML models in SeisBench using data from two earthquake swarms within the PNSN network:
1. The May 2017 earthquake swarm near Bremerton  
        - **Note** this example uses just 2 days of the swarm: May 10-11, 2017. Catalog searches show the swarm can broadly be defined between May 8 and November 2017 (potentially longer - a declustering analysis would better define this).  
![Event map of earthquakes in the seismic swarm near Bremerton, WA](data/2017-05-10_BRM/image.png)  


2. The August 23-24, 2013 earthquake swarm near Mount St. Helens


#### Attribution
- Code structure based on discussions with Yiyu Ni (University of Washington)  
- "Earthquake swarm NE of Bremerton" blog post by Renate Hartog on May 11, 2017  
https://www.pnsn.org/blog/2017/05/11/earthquake-swarm-ne-of-bremerton
- "Peppy seismic swarm 20 km NW of Mount St. Helens" blog post by John Vidale on August 24, 2013  
https://pnsn.org/blog/2013/08/24/peppy-seismic-swarm-20-km-nw-of-mount-st-helens

In [16]:
# Import dependencies
from seisbench.models import EQTransformer, PhaseNet
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os


In [46]:
def m2deg(x):
    return x/111139.

# Define fields for query
fields = ('starttime','endtime','latitude','longitude','maxradius')
# Define data selection kwargs 
dict_BRM_eve = dict(zip(fields,[UTCDateTime("2017-05-11T00:00:00"),\
                                UTCDateTime("2017-05-12T23:59:59"),\
                                47.5828,-122.57841,m2deg(7.5e3)]))
dict_BRM_sta = dict(zip(fields,[UTCDateTime("2017-05-10T00:00:00"),\
                                UTCDateTime("2017-05-11T23:59:59"),\
                                47.5828,-122.57841,m2deg(20e3)]))

# dict_MSH_sta = dict(zip(fields,[UTCDateTime("2017-05-11T00:00:00"),\
#                                 UTCDateTime("2017-05-12T23:59:59"),\
#                                 47.5828,-122.57841,\
#                                 7500.,50e3]))


# dict_MSH = dict(zip(fields,[]))

# Set desired query key-word arguments for subsequent blocks
e_kwargs = dict_BRM_eve
s_kwargs = dict_BRM_sta

print(s_kwargs)

{'starttime': UTCDateTime(2017, 5, 10, 0, 0), 'endtime': UTCDateTime(2017, 5, 11, 23, 59, 59), 'latitude': 47.5828, 'longitude': -122.57841, 'maxradius': 0.17995483133733434}


In [39]:
m2deg(50e3)

0.0022227799999999997

In [49]:
# Initialize Client
client = Client('IRIS')
# Download inventory with specified query down to channel 
# (ML generally operates on data w/o instrument response correction)
inv = client.get_stations(**s_kwargs,level='channel')


In [50]:
# Display Sensor Network
print(inv)

Inventory created at 2023-10-05T22:05:37.583300Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2017-05-...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (4):
			GS, NP, SY, UW
		Stations (48):
			GS.BAL (Seattle urban array)
			GS.CIN (Seattle urban array)
			GS.CTR (Seattle Center)
			GS.FINI (Seattle urban array)
			GS.LAU (Seattle urban array)
			GS.NOB (Seattle urban array)
			GS.PIE (Seattle urban array)
			GS.PIO2 (Seattle urban array)
			GS.SOC3 (Seattle urban array)
			GS.UNK (Seattle urban array)
			GS.WE2 (Seattle urban array)
			NP.7010 (WA:Seattle;Crowne Plaza Hotel)
			NP.7032 (WA:West Seattle;FS 29)
			NP.7037 (WA:Seattle;Ship Canal Locks)
			NP.7050 (WA:Seattle;New Fed Courthouse)
			NP.7051 (WA:Bremerton;New FS No. 1)
			SY.GNW (GNW synthetic)
			UW.ALKI (Alki Wastewater Plant, Seattle, WA, USA)
			UW.BABE (Bainbridge, WA, USA)
			UW.GMW (Gold Mountain, WA, USA)
			UW.GNW 

In [51]:
# Download catalog
cat = client.get_events(**e_kwargs)

In [53]:
cat

19 Event(s) in Catalog:
2017-05-12T02:02:56.860000Z | +47.607, -122.543 | 1.04 Ml
2017-05-11T22:05:13.400000Z | +47.605, -122.549 | 1.58 Ml
...
2017-05-11T01:49:33.960000Z | +47.607, -122.544 | 0.31 Md
2017-05-11T00:41:03.700000Z | +47.598, -122.546 | 0.98 Ml
To see all events call 'print(CatalogObject.__str__(print_all=True))'

In [ ]:
# Load PNSN catalog data from custom queries for Bremerton-Adjacent Swarm (BRM)
df_BRM_full = pd.read_csv(os.path.join('data','2017-05-10_BRM','pnsn_event_catalog_2017_BRM_cluster.csv'),\
                          parse_dates=['Time UTC'],index_col='Time UTC')
# Sort by index
df_BRM_full = df_BRM_full.sort_index()

print(df_BRM_full)

In [ ]:
# Plot the catalog
fig = plt.figure(figsize=(10,10))
axa = fig.add_subplot(211)
axa.plot(df_BRM_full['Magnitude'],'.')
axa.set_xlabel('UTC Date Time')
axa.set_ylabel('Magnitude (mixed methods)')
axb = axa.twinx()
axb.plot(df_BRM_full['Magnitude'].cumsum(),'r-',alpha=0.5)
axb.set_ylabel('Cumulative magnitude',labelpad=15)

# Define window to assess
TS = pd.Timestamp("2017-05-11")
TE = pd.Timestamp("2017-05-13T00:00:00")

axa.fill_between([TS,TE],[axa.get_ylim()[0]]*2,[axa.get_ylim()[1]]*2,color='yellow')


axc = fig.add_subplot(212)
IND = (df_BRM_full.index >= TS) & (df_BRM_full.index <= TE)
axc.plot(df_BRM_full[IND]['Magnitude'],'.')
axd = axc.twinx()
axd.plot((10**df_BRM_full[IND]['Magnitude']).cumsum(),'r-',alpha=0.5)

# axc = fig.add_subplot(223)
# ch = axc.scatter(df_BRM_full['Lon'],df_BRM_full['Lat'],s = 5**df_BRM_full['Magnitude'],c=df_BRM_full.index)
# plt.colorbar(ax=axc,cax=ch)

### Data selection
Use the following fields to define an event and station query kwargs for use with the `obspy`
 - **starttime**: [obspy.UTCDateTime] Start time of query in UTC
 - **endtime**:      [obspy.UTCDateTime] End time of query in UTC
- **latitude**:     [float] centroid latitude for queries in *degrees North*
 - **longitude**:    [float] centroid longitude for queries in *degrees East*
 - **minradius**: [float] minimum radius for query in *meters*
 - **Sradius**: [float] radius for Station query in *meters*

In [ ]:
# Use IRIS webservices / FDSN to query station, event, and waveform data 
